In [65]:
import pandas as pd
import numpy as np 
file_path = 'output/deg1/bernoulli/'

beta = [1,2]
B = [0.06, 0.06] 
probs = [[0.06, 0.25, 1/3, 2/3, 1], [0.06, 0.25, 1/3, 2/3, 1]]
design = "bernoulli"
n = 1000                  
nc = 50  
p_in = 0.5                 
p_out = (0.5-p_in)/49

for b in range(len(beta)):
    for p in probs[b]:
        file_path = 'output/deg' + str(beta[b]) + '/bernoulli/'
        fixed = '_n' + str(n) + '_nc' + str(nc) + '_' + 'in' + str(np.round(p_in,3)).replace('.','') + '_out' + str(np.round(p_out,3)).replace('.','') + '_p' + str(np.round(p,3)).replace('.','') + '_B' + str(B[b]).replace('.','')
        file_name = 'correlation' + fixed + '_deg' + str(beta[b]) + '_' + design + '-full.csv'
        
        df = pd.read_csv(file_path + file_name)
        
        df = df.drop(df[((df['Estimator'] == 'LS-Prop') | (df['Estimator'] == 'LS-Num')) & (df['design'] == 'Cluster')].index)
        df.loc[(df['Estimator']=='DM') & (df['design']=='Cluster'), 'Estimator'] = 'DM-C'
        df.loc[(df['Estimator']=='DM($0.75$)') & (df['design']=='Cluster'), 'Estimator'] = 'DM-C($0.75$)'
        
        df.to_csv(file_path + file_name)
        print("{} done".format(file_name))


'''
file_name = 'correlation_n1000_nc50_in05_out00_p1_B002_deg1_bernoulli-full.csv'

df = pd.read_csv(file_path + file_name)

# delete rows where (Estimator == LS-Prop or Estimator == LS-Num) and design == Cluster
df_new = df.drop(df[((df['Estimator'] == 'LS-Prop') | (df['Estimator'] == 'LS-Num')) & (df['design'] == 'Cluster')].index)

# where Estimator == DM and Design == Cluster, rename Estimator to DM-C
df_new.loc[(df['Estimator']=='DM') & (df['design']=='Cluster'), 'Estimator'] = 'DM-C'

# where Estimator == DM($0.75$) and Design == Cluster, rename Esimator to DM-C($0.75$)
df_new.loc[(df['Estimator']=='DM($0.75$)') & (df['design']=='Cluster'), 'Estimator'] = 'DM-C($0.75$)'
'''

correlation_n1000_nc50_in05_out00_p006_B006_deg1_bernoulli-full.csv done
correlation_n1000_nc50_in05_out00_p025_B006_deg1_bernoulli-full.csv done
correlation_n1000_nc50_in05_out00_p0333_B006_deg1_bernoulli-full.csv done
correlation_n1000_nc50_in05_out00_p0667_B006_deg1_bernoulli-full.csv done
correlation_n1000_nc50_in05_out00_p1_B006_deg1_bernoulli-full.csv done
correlation_n1000_nc50_in05_out00_p006_B006_deg2_bernoulli-full.csv done
correlation_n1000_nc50_in05_out00_p025_B006_deg2_bernoulli-full.csv done
correlation_n1000_nc50_in05_out00_p0333_B006_deg2_bernoulli-full.csv done
correlation_n1000_nc50_in05_out00_p0667_B006_deg2_bernoulli-full.csv done
correlation_n1000_nc50_in05_out00_p1_B006_deg2_bernoulli-full.csv done


"\nfile_name = 'correlation_n1000_nc50_in05_out00_p1_B002_deg1_bernoulli-full.csv'\n\ndf = pd.read_csv(file_path + file_name)\n\n# delete rows where (Estimator == LS-Prop or Estimator == LS-Num) and design == Cluster\ndf_new = df.drop(df[((df['Estimator'] == 'LS-Prop') | (df['Estimator'] == 'LS-Num')) & (df['design'] == 'Cluster')].index)\n\n# where Estimator == DM and Design == Cluster, rename Estimator to DM-C\ndf_new.loc[(df['Estimator']=='DM') & (df['design']=='Cluster'), 'Estimator'] = 'DM-C'\n\n# where Estimator == DM($0.75$) and Design == Cluster, rename Esimator to DM-C($0.75$)\ndf_new.loc[(df['Estimator']=='DM($0.75$)') & (df['design']=='Cluster'), 'Estimator'] = 'DM-C($0.75$)'\n"

In [ ]:
import numpy as np
def outcome_sums(n, Y, Z, selected):
    '''
    Returns the sums of the outcomes Y(z_t) for each timestep t

    Y (function): potential outcomes model
    Z (numpy array): treatment vectors z_t for each timestep t
    - each row should correspond to a timestep, i.e. Z should be beta+1 by n
    selected (list): indices of units in the population selected to be part of the experiment (i.e in U)
    '''
    sums, sums_U = np.zeros(Z.shape[0]), np.zeros(Z.shape[0]) 
    if len(selected) == n: # if we selected all nodes, sums = sums_U
        for t in range(Z.shape[0]):
            outcomes = Y(Z[t,:])
            sums[t] = np.sum(outcomes)
            sums_U[t] = np.sum(outcomes)
        return sums, sums_U
    else: 
        for t in range(Z.shape[0]):
            outcomes = Y(Z[t,:])
            sums[t] = np.sum(outcomes)
            sums_U[t] = np.sum(outcomes[selected])
    return sums, sums_U


In [66]:
import numpy as np
from scipy.sparse import csr_array
def binary_covariate_weights(n, nc, mu1, mu2, phi, A, seed=0):
    ''' Returns weighted adjacency matrix, where weights depend on a binary covariate type

    C[i,j] ~ Normal(mu1*mu1, 0.5) if both i and j are type 1
    C[i,j] ~ Normal(mu2*mu2, 0.5) if both i and j are type 2
    C[i,j] ~ Normal(mu1*mu2, 0.5) if i and j are different types

    Parameters
    ----------
    nc : int
        number of clusters
    mu1 : float

    mu2: float
        
    phi : float
        probability of switching type
    A : scipy sparse csr array
        adjacency matrix

    Returns
    ---------
    C : scipy sparse csr array
        weighted adjacency matrix
    '''
    n = A.shape[0]
    means = np.ones(n)
    midpoint = int((nc//2)*(n/nc))
    means[0:midpoint] = mu1
    means[midpoint:] = mu2

    rng  = np.random.default_rng(seed)
    switchers = rng.random(n)
    switchers = (switchers < phi) + 0 
    means = np.concatenate((np.where(switchers[0:midpoint]==1, mu2, means[0:midpoint]), np.where(switchers[midpoint:]==1, mu1, means[midpoint:])))
    means = np.outer(means, means)
    weights = rng.normal(loc=means, scale=0.5)

    return csr_array(A.multiply(weights))

In [69]:
n = 4
nc = 2
mu1 = 1/2
mu2  = 2.5
phi = 0
A = np.array([[1,0,1,0], [0,1,0,0], [0,1,1,0], [1,0,0,1]])
A = csr_array(A)
print(A.toarray())

C = binary_covariate_weights(n, nc, mu1, mu2, phi, A)
print()
print(C.toarray())

z = np.array([1,1,0,0])
z_sp = csr_array(z)

print()
print(C.dot(z))

print()
print(C * z_sp)


[[1 0 1 0]
 [0 1 0 0]
 [0 1 1 0]
 [1 0 0 1]]

[[-0.01783469  0.          1.90200002  0.        ]
 [ 0.         -0.38271074  0.          0.        ]
 [ 0.          1.14060417  5.62704453  0.        ]
 [ 0.97787051  0.          0.          6.77125668]]

[-0.01783469 -0.38271074  1.14060417  0.97787051]

  (0, 0)	-0.01783468658055548
  (1, 1)	-0.38271073552302626
  (2, 1)	1.140604168033727
  (3, 0)	0.977870508571345


## Testing Covariate Type (Binary) fucntion

In [ ]:
def covariate_type(i, n, K, num):
    '''
    Returns the covariate type of unit i. Assumes that the number of communities is divisible by the number of covariate types.

    n (int) = number of units in the population
    K (int) = number of communities (blocks) in the SBM
    num (int)  = number of covariate types

    Example: Suppose n=8, K=4, and num = 2. 
        Communities 0 and 1 are assigned covariate type 0 and contain individuals 0,1,2,3
        Communities 2 and 3 are assigned covariate type 1 and contain individuals 4,5,6,7
        Individual i's covariate type is i // 4. Note that 4 = n // num.

    Example: Suppose n=8, K=2, and num = 2.
        Community 0 is assigned covariate type 0 and contains individuals 0,1,2,3
        Communities 1 is assigned covariate type 1 and contains individuals 4,5,6,7
        Individual i's covariate type is i // 4. Note that 4 = n // num.

    Example: Suppose n=8, K=4, and num = 4.
        Community 0 is assigned covariate type 0 and contains individuals 0,1
        Community 1 is assigned covariate type 1 and contains individuals 2,3
        Community 2 is assigned covariate type 2 and contains individuals 4,5
        Community 3 is assigned covariate type 3 and contains individuals 6,7
        Individual i's covariate type is i // 2. Note that 2 = n // num.
    '''
    assert num <= K and num%K==0, "there cannot be more covariate types than number of communities; number of types should divide evenly into the number of groups"
    div = n // num
    return i // div

In [ ]:
n = 25
K = 5
num = 5

for i in range(n):
    print(covariate_type(i, n, K, num))

In [ ]:
import numpy as np
C = np.zeros((5,5))
print(C)

In [ ]:
arr = np.zeros(10)
arr[0:10//2] = 1
print(arr)
arr[10//2:] = 2
print(arr)

In [ ]:
x1 = np.arange(9.0).reshape((3, 3))
x2 = np.arange(3.0)
print(x1)
print()
print(x2)
print()
print(np.multiply(x1, x2))

In [ ]:
arr = np.array([0,0,0,0,1,1,1,1])
P = np.random.rand(8)
print(arr)
print(P)

In [ ]:
P = (P < 0.5)+0
print(P)
np.nonzero(P)

In [ ]:
arr = np.zeros(8) + 0.2
print(arr)

arr[np.nonzero(P)] = 0.8
print(arr)

In [ ]:
from scipy import sparse
sA = sparse.csr_matrix(arr)
print(sA, sA.shape)

In [ ]:
B = np.array([2, 3, 1])
arrB = np.multiply(arr,B)
print(arrB, arrB.shape)

In [ ]:
AB = sA.multiply(B)
print(AB, AB.shape)

## Horvitz-Thompson

In [ ]:
n = 1000
nc = 50
inds = [0, 10, 19, 20, 38, 39, 40, 59, 60]
lst = len(set([i//(n/nc) for i in inds]))
lst

In [ ]:
x = np.array([[3, 0, 0], [0, 4, 0], [5, 6, 0]])
x

In [ ]:
import numpy as np

n = 8
nc = 2
# Replace this with your actual numpy array
A = random_array = np.random.randint(2, size=(n, n))
print(A)
print()

# Get the indices of nonzero elements for each row
nonzero_indices = [list(np.nonzero(row)[0]) for row in A]
print(nonzero_indices)
print()

# list of cluster neighborhood sizes
newList = [len(set([i // (n/nc) for i in neighbors])) for neighbors in nonzero_indices]
print(newList)
print()

np.power(0.5, newList)

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix

# Replace this with your actual sparse matrix
data = np.array([1, 0, 3, 0, 5, 0, 6, 0, 8])
row_indices = np.array([0, 0, 0, 1, 1, 0, 2, 2, 2])
col_indices = np.array([0, 2, 2, 0, 2, 0, 2, 2, 0])

A = csr_matrix((data, (row_indices, col_indices)), shape=(3, 3))

# Get the indices of nonzero elements for each row
nonzero_indices = [list(row.nonzero()[1]) for row in A]

print(nonzero_indices)

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix

# Replace these with your actual arrays and dimensions
n = 3  # Adjust the size of the arrays as needed
data_z1 = np.array([[1, 0, 1], [0, 1, 0], [1, 0, 1]])
z1 = csr_matrix(data_z1)
z2 = np.array([10, 20, 30])

# Find the nonzero indices for each row in z1
neighborhoods = [list(row.nonzero()[1]) for row in A] # list of lists of nonzero indices in each row of A
print(neighborhoods)
print()

neighbor_treatments = [list(z2[neighborhood]) for neighborhood in neighborhoods]
print(neighbor_treatments)

all_treated = [np.prod(treatments) for treatments in neighbor_treatments]
none_treated = [all(z == 0 for z in treatments)+0 for treatments in neighbor_treatments]

In [ ]:
import scipy
# Replace this with your actual sparse matrix
data = [[1, 0, 3], [0, 5, 0], [6, 0, 8]]
sparse_matrix = scipy.sparse.csr_array(data)

# Calculate the sum of each row
row_sums = sparse_matrix.sum(axis=1).tolist()
print(row_sums)

In [1]:
from scipy.sparse import csr_matrix
def horvitz_thompson(n, nc, y, A, z, q, p):
    '''Computes the Horvitz-Thompson estimate of the TTE under Bernoulli design or Cluster-Bernoulli design.
    
    Parameters
    ----------
    n : int
        the size of the population/network
    nc : int
        the number of clusters (equals n if simple Bernoulli design with no clustering)
    y : numpy array
        the outcomes of each unit in the population
    A : scipy sparse array
        adjacency matrix of the network such that A[i,j]=1 indicates that unit j is an in-neighbor of i
    z : numpy array
        the treatment assignment of each unit in the population
    q : float
        probability that a cluster is indepdently chosen for treatment (should equal 1 under simple Bernoulli design with no clustering)
    p : float
        the treatment probability for chosen clusters in the staggered rollout
    '''

    neighborhoods = [list(row.nonzero()[1]) for row in A] # list of neighbors of each unit
    neighborhood_sizes = A.sum(axis=1).tolist() # size of each unit's neighborhood
    neighbor_treatments = [list(z[neighborhood]) for neighborhood in neighborhoods] # list of treatment assignments in each neighborhood

    A = A * csr_matrix(np.diag(np.repeat(np.arange(1,nc+1),n//nc))) # modifies the adjancecy matrix so that if there's an edge from j to i, A[i,j]=cluster(j)
    cluster_neighborhoods = [np.unique(row.data,return_counts=True) for row in A] # for each i, cluster_neighborhoods[i] = [a list of clusters i's neighbors belong to, a list of how many neighbors are in each of these clusters]
    cluster_neighborhood_sizes = [len(x[0]) for x in cluster_neighborhoods] # size of each unit's cluster neighborhood
    
    # Probabilities of each person's neighborhood being entirely treated or entirely untreated
    all_treated_prob = np.multiply(np.power(p, neighborhood_sizes), np.power(q, cluster_neighborhood_sizes))
    none_treated_prob = [np.prod((1+q) + np.power(1-p, x[1])*q) for x in cluster_neighborhoods]
    
    # Indicators of each person's neighborhood being entirely treated or entirely untreated
    all_treated = [np.prod(treatments) for treatments in neighbor_treatments]
    none_treated = [all(z == 0 for z in treatments)+0 for treatments in neighbor_treatments]

    zz = np.nan_to_num(np.divide(all_treated,all_treated_prob) - np.divide(none_treated,none_treated_prob))

    return 1/n * y.dot(zz)

In [1]:
from experiment_functions import *
import numpy as numpy
def run_experiment(beta, n, nc, B, r, diag, Pii, Pij, phi, design, q_or_K, graphNum, T):
    '''
    beta = degree of the model / polynomial
    n = population size
    nc = number of clusters
    B = original treatment budget/fraction
    r = ratio offdiag/diag: (indirect effect)/(direct effects)
    diag = maxium norm of the direct effects before covariate type scaling
    Pii = edge probability within communities
    Pij = edge prob btwn different communities
    phi = correlation btwn community & effect type (probability between 0 and 0.5)
    design = design being used for selecting clusters, either "complete" or "bernoulli"
    q_or_K = if using complete RD for selecting cluster, this will be the value of K; if using Bernoulli design, this will be the value q
    graphNum = number of graphs to average over
    T = number of trials per graph
    graphStr = type of graph e.g. "SBM" for stochastic block model or "ER" for Erdos-Renyi
    p_prime = the budget on the boundary of U
    '''
    
    offdiag = r*diag   # maximum norm of indirect effect

    if design == "complete":
        K = q_or_K
        q = K/nc
    else:
        q = q_or_K
        K = int(np.floor(q*nc))

    p = B/q
    G, A = SBM(n, nc, Pii, Pij)  #returns the SBM networkx graph object G and the corresponding adjacency matrix A

    for g in range(graphNum):
        # random weights for the graph edges
        rand_wts = np.random.rand(n,3)
        alpha = rand_wts[:,0].flatten()
        C = simpleWeights(A, diag, offdiag, rand_wts[:,1].flatten(), rand_wts[:,2].flatten())
        C = covariate_weights_binary(C, minimal = 1/4, extreme = 4, phi=phi)
        
        # potential outcomes model
        fy = ppom(beta, C, alpha)

        # compute the true TTE
        TTE = 1/n * np.sum((fy(np.ones(n)) - fy(np.zeros(n))))
        print("TTE: {}\n".format(TTE))
        
        ####### Estimate ########

        # parameters for the staggered rollout - Cluster Randomized Design
        P = seq_treatment_probs(beta, p)        # treatment probabilities for each step of the staggered rollout on U
        P_prime = seq_treatment_probs(beta, 0)  # treatment probabilities for each step of the staggered rollout on the boundary of U

        TTE_ht = np.zeros(T)
        for i in range(T):
            # select clusters 
            if design == "complete":
                selected = select_clusters_complete(nc, K)
            else:
                selected = select_clusters_bernoulli(nc, q)
            
            # U
            selected_nodes = [x for x,y in G.nodes(data=True) if (y['block'] in selected)] # get the nodes in selected clusters

            # Cluster Randomized Design
            Z = staggered_rollout_bern_clusters(n, selected_nodes, P, [], P_prime)
            z = Z[beta,:]
            y = fy(z)

            TTE_ht[i] = horvitz_thompson(n, nc, y, A, z, q, p)

        print("H-T: {}".format(np.sum(TTE_ht)/T))
        print("H-T MSE: {}\n".format(np.sum((TTE_ht-TTE)**2)/T))

def covariate_weights_binary(C, minimal = 1/4, extreme = 4, phi=0):
    '''
    Returns a weighted adjacency matrix where weights are determined by covariate type. We assume a binary effect types covariate

    C (array): weights without effect type covariate
    minimal (float): 
    extreme (int):
    phi: probability that an individual's covariate type flips
    '''
    n = C.shape[0]
    scaling1 = np.zeros(n//2) + minimal
    scaling2 = np.zeros(n//2) + extreme

    R1 = np.random.rand(n//2)
    R2 = np.random.rand(n//2)
    R1 = (R1 < phi) + 0
    R2 = (R2 < phi) + 0

    scaling1[np.nonzero(R1)] = extreme
    scaling2[np.nonzero(R2)] = minimal

    scaling = np.concatenate((scaling1,scaling2))
    return C.multiply(scaling)

def SBM(n, k, Pii, Pij):
    '''
    Returns the adjacency matrix of a stochastic block model on n nodes with k communities
    The edge prob within the same community is Pii
    The edge prob across different communities is Pij
    '''
    sizes = np.zeros(k, dtype=int) + n//k
    probs = np.zeros((k,k)) + Pij
    np.fill_diagonal(probs, Pii)
    G = nx.stochastic_block_model(sizes, probs)
    A = nx.adjacency_matrix(nx.stochastic_block_model(sizes, probs))
    #blocks = nx.get_node_attributes(G, "block")
    return G, A

In [3]:
beta = 1
n = 1000
nc = 50
B = 0.06
p = 1
r = 1.25
diag = 1
Pii = 10/(n/nc)
Pij = 0
phi = 0
#design = "complete"
#q_or_K = int(np.floor(B * nc / p))
design = "bernoulli"
q_or_K = 0.6
graphNum = 1
T = 100

run_experiment(beta, n, nc, B, r, diag, Pii, Pij, phi, design, q_or_K, graphNum, T)

TTE: 2.528946140581735

H-T: -0.33747227618894143
H-T MSE: 8.216938497017704



In [ ]:
all_treated = [0, 1, 0, 0, 0]
all_treated_prob = [0, 0.25, 0.75, 0.25, 0]
none_treated = [0, 0, 1, 0, 1]
none_treated_prob = [0, 0.25, 0.25, 0.5, 0.75]
arr1 = np.divide(all_treated,all_treated_prob)
arr2 = np.divide(none_treated,none_treated_prob)
arr3 = arr1 - arr2
print(arr1)
print()
print(arr2)
print()
print(arr3)
np.nan_to_num(arr1, copy=False)
np.nan_to_num(arr2, copy = False)
print(arr1)
print()
print(arr2)
print()
np.nan_to_num(arr3, copy = False)
print(arr3)

In [ ]:
n = 1000
nc = 50
l = 1

cluster = set(range(l*(n//nc), (l+1)*(n//nc)))
neighbors = set([20, 25, 40, 62])

len(cluster & neighbors)

for c in cluster:
    print(c)

In [12]:
import numpy as np
from scipy.sparse import random, csr_matrix

n = 10
nc = 5

random_sparse_matrix = random(n, n, density=0.5, format='csr')
random_sparse_matrix.data[:] = 1
print(random_sparse_matrix.toarray())


[[1. 1. 0. 1. 1. 1. 0. 0. 0. 0.]
 [1. 0. 1. 1. 1. 1. 1. 1. 0. 0.]
 [1. 1. 1. 0. 1. 1. 0. 1. 1. 1.]
 [0. 0. 1. 0. 1. 0. 0. 0. 1. 0.]
 [0. 0. 1. 1. 0. 1. 0. 1. 1. 0.]
 [1. 1. 1. 0. 0. 1. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 1. 0. 1.]
 [1. 0. 0. 0. 1. 1. 0. 0. 1. 1.]
 [1. 0. 0. 1. 0. 0. 0. 0. 1. 0.]
 [1. 1. 0. 1. 0. 1. 1. 0. 0. 0.]]


In [13]:
sp = csr_matrix(np.diag(np.repeat(np.arange(1,nc+1),n//nc)))
print(sp.toarray())

#clusters = np.arange(1,nc+1)
#clusters = np.repeat(clusters,n//nc)
#clusters = np.tile(np.repeat(np.arange(1,nc+1),n//nc), (n,1))

#myprod = random_sparse_matrix.multiply(np.tile(np.repeat(np.arange(1,nc+1),n//nc), (n,1)))
myprod2 = random_sparse_matrix * sp

print()
print(myprod2.toarray())

[[1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 2 0 0 0 0 0 0 0]
 [0 0 0 2 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 3 0 0 0 0]
 [0 0 0 0 0 0 4 0 0 0]
 [0 0 0 0 0 0 0 4 0 0]
 [0 0 0 0 0 0 0 0 5 0]
 [0 0 0 0 0 0 0 0 0 5]]

[[1. 1. 0. 2. 3. 3. 0. 0. 0. 0.]
 [1. 0. 2. 2. 3. 3. 4. 4. 0. 0.]
 [1. 1. 2. 0. 3. 3. 0. 4. 5. 5.]
 [0. 0. 2. 0. 3. 0. 0. 0. 5. 0.]
 [0. 0. 2. 2. 0. 3. 0. 4. 5. 0.]
 [1. 1. 2. 0. 0. 3. 4. 0. 5. 0.]
 [0. 0. 0. 0. 0. 3. 0. 4. 0. 5.]
 [1. 0. 0. 0. 3. 3. 0. 0. 5. 5.]
 [1. 0. 0. 2. 0. 0. 0. 0. 5. 0.]
 [1. 1. 0. 2. 0. 3. 4. 0. 0. 0.]]


In [14]:
print(type(random_sparse_matrix))

<class 'scipy.sparse._csr.csr_matrix'>


In [16]:
from scipy.sparse import csr_array
G,A = SBM(10, 2, 2/(10/2), 0)
print(A.toarray(), type(A))

temp = csr_array(np.tile(np.repeat(np.arange(1,nc+1),n//nc), (n,1)))#csr_array(np.diag(np.repeat(np.arange(1,nc+1),n//nc)))
print(temp.toarray())

my = A.multiply(temp) #A * temp
print(my.toarray())


[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 0 1]
 [0 0 0 0 0 1 0 1 1 1]
 [0 0 0 0 0 1 1 0 1 0]
 [0 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 1 1 0 0 0]] <class 'scipy.sparse._arrays.csr_array'>


In [14]:
#np.prod(A*z,axis=1)

A = np.array([[1,0,0,1,1], [0,1,0,1,0], [0,0,1,1,1], [0,0,0,1,0], [1,0,1,0,1]])
z = np.array([0,0,1,1,1])
print(A)
print()
print(z)
A&z

[[1 0 0 1 1]
 [0 1 0 1 0]
 [0 0 1 1 1]
 [0 0 0 1 0]
 [1 0 1 0 1]]

[0 0 1 1 1]


array([[0, 0, 0, 1, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 1, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 1]])

In [24]:
n = 10
nc = 2
avg = 2
G,A = SBM(n, nc, avg/(n/nc), 0)
z = np.repeat([1, 0], 5)
print(A.toarray())
print()

neighborhoods = [list(row.nonzero()[1]) for row in A] # list of neighbors of each unit
neighborhood_sizes = A.sum(axis=1).tolist() # size of each unit's neighborhood
neighbor_treatments = [list(z[neighborhood]) for neighborhood in neighborhoods] # list of treatment assignments in each neighborhood

A = A.multiply(csr_array(np.tile(np.repeat(np.arange(1,nc+1),n//nc), (n,1)))) # modifies the adjancecy matrix so that if there's an edge from j to i, A[i,j]=cluster(j)
print(A.toarray())
cluster_neighborhoods = [np.unique(row.data,return_counts=True) for row in A] # for each i, cluster_neighborhoods[i] = [a list of clusters i's neighbors belong to, a list of how many neighbors are in each of these clusters]
cluster_neighborhood_sizes = [len(x[0]) for x in cluster_neighborhoods] # size of each unit's cluster neighborhood


[[0 0 1 1 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [1 0 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [1 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 1 0 0 1 1]
 [0 0 0 0 0 0 1 1 0 1]
 [0 0 0 0 0 0 1 1 1 0]]

[[0 0 1 1 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [1 0 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [1 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 0 2 2]
 [0 0 0 0 0 2 0 0 2 2]
 [0 0 0 0 0 0 2 2 0 2]
 [0 0 0 0 0 0 2 2 2 0]]


In [28]:
for x in cluster_neighborhoods:
    print(x[0], x[1])

[1] [3]
[1] [1]
[1] [2]
[1] [1]
[1] [3]
[2] [1]
[2] [2]
[2] [3]
[2] [3]
[2] [3]


In [29]:
cluster_neighborhood_sizes

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [2]:
import pandas as pd
import numpy as np
datatypes = {'Estimator': str, 'Bias': np.float64, 'Abs_Bias': np.float64, 'Rel_bias_sq': np.float64, 'Bias_sq': np.float64}
df = pd.read_csv('output/deg1/SBM_n1000_nc50_in04_out0002_p1_B006_correlation-full-data_deg1.csv')

In [55]:
#{'a': np.float64, 'b': np.int32, 'c': 'Int64'} str 
#Bias,Abs_Bias,Rel_bias_sq,Bias_sq

datatypes = {'Estimator': str, 'Bias': np.float64, 'Abs_Bias': np.float64, 'Rel_bias_sq': np.float64, 'Bias_sq': np.float64}

Index(['Unnamed: 0', 'n', 'nc', 'Pii', 'Pij', 'Phi', 'K', 'p', 'q', 'B',
       'ratio', 'out-in', 'Graph', 'rep', 'design', 'Estimator', 'Bias',
       'Abs_Bias', 'Rel_bias_sq', 'Bias_sq'],
      dtype='object')

In [57]:
df.loc[:,'Estimator']

0                   PI-$n$($p$)
1         PI-$\mathcal{U}$($p$)
2                            HT
3                       LS-Prop
4                        LS-Num
                  ...          
118795              PI-$n$($B$)
118796                  LS-Prop
118797                   LS-Num
118798                       DM
118799               DM($0.75$)
Name: Estimator, Length: 118800, dtype: object

In [5]:
isinstance([5.234567], list)

True